In [1]:
#Lab 3
#Housekeeping
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm

from statsmodels.stats.outliers_influence \
     import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
!pip install ISLP
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)



  DEPRECATION: Building 'autograd-gamma' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'autograd-gamma'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/3.6 MB ? eta -:--:--
   ---------------------------------------- 3.6/3.6 MB 28.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/832.4 kB ? eta -:--:--
   --------------------------------------- 832.4/832.4 kB 21.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   - -------------------------------------- 7.3/241.3 MB 34.7 MB/s eta 0:00:07
   -- ------------------------------------- 14.9/241.3 MB 35.3 MB/s eta 0:00:07
   --- ------------------------------------ 22.5/241.3 MB 35.1 MB/s eta 0:00:07
   ---- ----------------------------------- 29.4/241.3 MB 34.6 MB/s eta 0:00:07
   ------ --------------------------------- 36.4/241.3 MB 34.4 MB/s eta 0:00:06
   ------- -------------------------------- 43.8/241.3 MB 34.7 MB/s eta 0:00:06
   -------- ------------------------------- 51.4/24

In [6]:
#Question 1

College = load_data('College')

College.head(10)


,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
5,Yes,587,479,158,38,62,678,41,13500,3335,500,675,67,73,9.4,11,9727,55
6,Yes,353,340,103,17,45,416,230,13290,5720,500,1500,90,93,11.5,26,8861,63
7,Yes,1899,1720,489,37,68,1594,32,13868,4826,450,850,89,100,13.7,37,11487,73
8,Yes,1038,839,227,30,63,973,306,15595,4400,300,500,79,84,11.3,23,11644,80
9,Yes,582,498,172,21,44,799,78,10468,3380,660,1800,40,41,11.5,15,8991,52


In [7]:
#Question 1 (Cont.)
College.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Private      777 non-null    category
 1   Apps         777 non-null    int64   
 2   Accept       777 non-null    int64   
 3   Enroll       777 non-null    int64   
 4   Top10perc    777 non-null    int64   
 5   Top25perc    777 non-null    int64   
 6   F.Undergrad  777 non-null    int64   
 7   P.Undergrad  777 non-null    int64   
 8   Outstate     777 non-null    int64   
 9   Room.Board   777 non-null    int64   
 10  Books        777 non-null    int64   
 11  Personal     777 non-null    int64   
 12  PhD          777 non-null    int64   
 13  Terminal     777 non-null    int64   
 14  S.F.Ratio    777 non-null    float64 
 15  perc.alumni  777 non-null    int64   
 16  Expend       777 non-null    int64   
 17  Grad.Rate    777 non-null    int64   
dtypes: category(1), float64(1), in

#### Question 1 Discussion

This dataset provides statistics about a number of US universities from the year 1995. 'Outstate' describes the out of state tuition for a given observation (university), 'Private' is a boolean indicator variable describing if the college is private or public (Yes/No). 'Room.Board' describes the room and board costs for a given institution, 'PhD' describes the percent of faculty at a given university who hold a Ph.D., and Top10Perc describes the percentage of new students in the university who graduated in the top 10% of their Highschool class.

In [10]:
#Question 2
X = pd.DataFrame({'intercept': np.ones(College.shape[0]),
                  'Top10perc': College['Top10perc']})
y = College['Outstate']
model = sm.OLS(y, X).fit()
summarize(model)



,coef,std err,t,P>|t|
intercept,6906.4586,221.614,31.164,0.0
Top10perc,128.2437,6.774,18.931,0.0


#### Question 2 Discussion

The coefficient of Top10Perc on Outstate is 128.2437. The t statistic is 18.931, and the p value is 0.0, indicating that $\beta$<sub>Top10Perc</sub> is statistically significant at the 5% level. In other words, each additional % of new students at a university who graduated in the top 10% of their highschool class is associated with a $128.24 increase in the out of state tuition.

In [14]:
#Question 3
X = pd.DataFrame({'intercept': np.ones(College.shape[0]),
                  'Top10perc': College['Top10perc'],
                  'Room.Board': College['Room.Board'],
                  'PhD': College['PhD']})
y = College['Outstate']
model = sm.OLS(y, X).fit()
summarize(model)


,coef,std err,t,P>|t|
intercept,-430.6350,538.834,-0.799,0.424
Top10perc,82.0003,6.728,12.189,0.000
Room.Board,1.8825,0.097,19.397,0.000
PhD,5.6226,7.147,0.787,0.432


#### Question 3 Discussion
 In this model, the coefficient of Top10Perc is 82.0003, with a t statistic of 12.189 and a p value of 0. This indicates that $\beta$<sub>Top10Perc</sub> is significant at the 5% level, and that each new student at a given university who graduated in the top 10% of their highschool class is associated with an $82 increase in out of state tuition.

 The coefficient of Room.Board is 1.8825, with a t statistic of 19.397 and a p value of 0. This indicates that $\beta$<sub>Room.Board</sub> is statistically significant at the 5% level, and that each additional dollar of room and board charges are associated with a $1.88 increase in out of state tuition.

 The coefficient of Phd is 5.6226, with a t statistic of 0.787, and a p value of 0.432. This indicates that $\beta$<sub>PhD</sub> is not statistically significant at the 5% level, and that the percentage of Phd faculty at a given university is not associated with an increase or decrease in out of state tuition.

In [13]:
# Question 4
X = pd.DataFrame({'intercept': np.ones(College.shape[0]),
                  'Top10perc': College['Top10perc'],
                  'Top10Perc^2': College['Top10perc']**2,
                  'Room.Board': College['Room.Board'],
                  'PhD': College['PhD']})
y = College['Outstate']
model2 = sm.OLS(y, X).fit()
summarize(model2)

,coef,std err,t,P>|t|
intercept,-1087.2758,573.149,-1.897,0.058
Top10perc,138.0675,18.661,7.399,0.000
Top10Perc^2,-0.6859,0.213,-3.218,0.001
Room.Board,1.9081,0.097,19.712,0.000
PhD,1.9617,7.194,0.273,0.785


#### Question 4 Discussion

 In this model, the coefficient of Top10Perc is 138.0675, with a t statistic of 7.399 and a p value of 0. The quadratic term for Top10Perc has a coefficient of -0.6859 with a t statistic of -3.218 and a p value of 0.001. This indicates that $\beta$<sub>Top10Perc</sub> and $\beta$<sub>Top10Perc^2</sub> are significant at the 5% level, and that each new student at a given university who graduated in the top 10% of their highschool class is associated with a $138 + $(-0.6859*$n^2$<sub>Top10Perc</sub>) increase in out of state tuition.

 The coefficient of Room.Board is 1.9081, with a t statistic of 19.712 and a p value of 0. This indicates that $\beta$<sub>Room.Board</sub> is statistically significant at the 5% level, and that each additional dollar of room and board charges are associated with a $1.91 increase in out of state tuition.

 The coefficient of Phd is 1.9617, with a t statistic of 0.273, and a p value of 0.785. This indicates that $\beta$<sub>PhD</sub> is not statistically significant at the 5% level, and that the percentage of Phd faculty at a given university is not associated with an increase or decrease in out of state tuition.


In [16]:
anova_lm(model, model2)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,773.0,5.693394e+09,0.0,NaN,NaN,NaN
1,772.0,5.618020e+09,1.0,7.537388e+07,10.357499,0.001343


#### Question 5 Discussion
 For the linear model, the F stat is not reported. The F stat for the quadratic model is 10.357499, and the p value is 0.001343. This F stat and p value indicate that this model is statistically significant at the 5% level, and is therefore likely an improvement over the linear model.